```{contents}
```
## **Tool Node in LangGraph**

A **Tool Node** in LangGraph is a specialized execution node whose responsibility is to **invoke external tools or APIs** and integrate their outputs back into the graph’s shared state.
It enables LLM workflows to move beyond pure text generation and **interact with real systems** such as databases, search engines, code runtimes, and enterprise services.

---

### **1. Conceptual Role of a Tool Node**

In an agentic system, the LLM decides **what should be done**, but a Tool Node performs **the actual action**.

```
LLM Reasoning → Tool Node → Environment → Updated State → Next Decision
```

This separation creates:

* **Interpretability** (clear boundary between reasoning and acting)
* **Safety** (tools are sandboxed)
* **Reliability** (deterministic tool execution)

---

### **2. Tool Node in the Execution Graph**

| Component | Role                                           |
| --------- | ---------------------------------------------- |
| LLM Node  | Decides which tool to use and with what inputs |
| Tool Node | Executes the selected tool                     |
| State     | Stores tool inputs and outputs                 |
| Edges     | Route execution before and after tool use      |

---

### **3. Basic Tool Node Example**

```python
from langgraph.graph import StateGraph
from typing import TypedDict

class State(TypedDict):
    query: str
    result: str

def search_tool(state: State):
    # Simulated API call
    return {"result": f"Results for: {state['query']}"}

builder = StateGraph(State)

builder.add_node("tool_search", search_tool)
builder.set_entry_point("tool_search")

graph = builder.compile()
graph.invoke({"query": "LangGraph"})
```

---

### **4. LLM + Tool Workflow Pattern**

A typical pattern:

```
User → LLM Node → Tool Node → LLM Node → Final Output
```

The LLM produces a structured decision:

```json
{
  "tool": "search",
  "input": "LangGraph tool node"
}
```

Then the Tool Node executes it.

---

### **5. Production-Grade Tool Node with Routing**

```python
def router(state):
    if state["action"] == "search":
        return "search_tool"
    return "final"

builder.add_node("router", router)
builder.add_node("search_tool", search_tool)
builder.add_node("final", lambda s: s)

builder.add_conditional_edges("router", lambda s: s["action"], {
    "search": "search_tool",
    "final": "final"
})
```

---

### **6. Tool Node Variants**

| Variant         | Purpose                     |
| --------------- | --------------------------- |
| API Tool        | REST / GraphQL calls        |
| DB Tool         | SQL / NoSQL operations      |
| Search Tool     | Web or internal search      |
| Compute Tool    | Python / ML inference       |
| File Tool       | Read/write files            |
| System Tool     | OS / shell commands         |
| Cloud Tool      | AWS, GCP, Azure operations  |
| Enterprise Tool | CRM, ERP, internal services |

---

### **7. Safety & Governance for Tool Nodes**

| Mechanism         | Function                   |
| ----------------- | -------------------------- |
| Input validation  | Prevent malformed requests |
| Output validation | Enforce schema             |
| Sandboxing        | Isolate execution          |
| Rate limiting     | Prevent abuse              |
| Human approval    | High-risk operations       |
| Audit logging     | Compliance & traceability  |

---

### **8. Why Tool Nodes Matter**

Without Tool Nodes, LLM systems are **closed worlds**.
With Tool Nodes, they become **operational systems** capable of:

* Taking actions
* Retrieving real data
* Executing business logic
* Integrating with enterprise infrastructure

---

### **9. Mental Model**

> **LLM = Brain**
> **Tool Node = Hands**

LangGraph’s Tool Nodes give LLMs the ability to **act safely, deterministically, and at scale**.

---

If you want next, I can explain:

• How LLMs decide which tool to call
• Advanced tool orchestration patterns
• Security design for production tool usage
• Multi-tool agent architectures


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    user_input: str
    action: str
    tool_input: str
    tool_output: str
    final_answer: str

def reasoning_node(state: State):
    if "weather" in state["user_input"].lower():
        return {
            "action": "call_tool",
            "tool_input": state["user_input"]
        }
    else:
        return {
            "action": "final",
            "final_answer": "I can answer directly."
        }

def weather_tool(state: State):
    query = state["tool_input"]
    result = f"Weather data for: {query} = 30°C, Clear"
    return {"tool_output": result}

def final_node(state: State):
    if state.get("tool_output"):
        answer = f"Here is what I found: {state['tool_output']}"
    else:
        answer = state["final_answer"]
    return {"final_answer": answer}


In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("reason", reasoning_node)
builder.add_node("tool", weather_tool)
builder.add_node("final", final_node)

builder.set_entry_point("reason")

builder.add_conditional_edges(
    "reason",
    lambda s: s["action"],
    {
        "call_tool": "tool",
        "final": "final"
    }
)

builder.add_edge("tool", "final")
builder.add_edge("final", END)

graph = builder.compile()


In [3]:
result = graph.invoke({"user_input": "What is the weather in Delhi?"})
print(result["final_answer"])


Here is what I found: Weather data for: What is the weather in Delhi? = 30°C, Clear
